## Model 4

### 4.3 Fine tuning with pretrained network (VGG16)

In fine tuning method, we cancel freezing for few last part of conv layers, remaining first part which has general feature that 

can be re-used. Even, that we cancel freezing all parts of conv layers means that model have lots of parameters and it could lead 

our model to be overfitting.

In this part

1. Add new network on pretrained network (base_layer)
2. Freeze base_layer
3. Train new network
4. Cancel freezing for few part of base_layer
5. Train newnetwork with unfreezed base_layer

##### Data

In [1]:
import os

dir_path = 'C:/python'
base_dir = (dir_path + '/dogs-vs-cats_small')
train_dir = os.path.join(base_dir, 'train')
dev_dir = os.path.join(base_dir,'dev')
test_dir = os.path.join(base_dir, 'test')

##### Network

In [2]:
from keras import models
from keras import layers
from keras.layers import Dense, Activation, BatchNormalization, Dropout
from keras import optimizers
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

conv_base.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

##### Model

In [3]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


I am going to fine tune for last 3 networks, block5_conv1, block5_conv2, block5_conv3 and let other layers freeze

##### Freeze and unfreeze

In [4]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable=True
    
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [6]:
print(len(model.trainable_weights))

10


In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 9,177,089
Non-trainable params: 7,635,264
_________________________________________________________________


##### Compile & Training

With same generator in 3 and 4_convnet_with_pretrained_network, we can train this model.